# Import Resource Metadata

This notebook provides an example for uploading table descriptions to Secoda. To get started, you will need to [obtain an API key](https://app.secoda.co/settings/api)

Input your API key and Secoda API endpoint URL below. The URL for the cloud instance is `https://api.secoda.co`. If you are self-hosting Secoda or on the EU instance, you will have to update variable.

In [1]:

import requests

API_KEY = "<api_key>"
SECODA_API_URL = "https://api.secoda.co"

session = requests.Session()
session.headers.update(dict(
    Authorization=f"Bearer {API_KEY}"
))


def build_url(url: str):
    return f"{SECODA_API_URL}{url}"

## Parse your data

If your data is in CSV format, you could parse it this way and standardize the column names in there to match up with Secoda

In [ ]:
import pandas

dataframe = pandas.read_csv("data/entity_documentation.csv")
dataframe

## Upload documentation to Secoda

In [ ]:

def find_secoda_entity(entity_type, title, parent, *args, **kwargs):
    res = session.get(
        build_url(f"/table/{entity_type}s/?title={title}&parent={parent}")
    )
    results = res.json().get("results")
    if len(results) >= 1:
        return results[0]
    return None

def set_entity_documentation(entity_type, unique_id, description, documentation):
    session.put(
        build_url(f"/table/{entity_type}s/{unique_id}"),
        json=dict(
            description=description,
            definition=documentation
        )
    )

for index, entry in enumerate(dataframe.to_dict("records")):
    print(f"Processing {index}/{dataframe.shape[0]}")
    table = find_secoda_entity(
        entity_type=entry["entity_type"],
        title=entry["title"],
        parent=entry["parent"]
    )
    if table is not None:
        set_entity_documentation(entry["entity_type"], table.get("id"), entry.get("description", ""), entry.get("definition", ""))